# ZADANIA PROLOG IGOR FLIS 320530
Rozwiazania do zadań umieściłem poniżej, przy każdej (według mnie) istotej informacji dodawałem notatkę po % aby wskazać co miałem na myśli robiąc dane zadanie.

Z poważaniem *Igor Flis*

### Zadanie 1: Drzewo genealogiczne
**Zadanie**: Stwórz bazę faktów dla drzewa genealogicznego, która obejmuje relacje rodzic-dziecko, małżeństwo oraz płeć. Następnie zdefiniuj reguły `przodek(X, Y)`, `brat(X, Y)`, `siostra(X, Y)`, `kuzyn(X, Y)`.

In [ ]:
% Dyrektywy discontiguous
:- discontiguous ojciec/2.
:- discontiguous matka/2.
:- discontiguous malzenstwo/2.
:- discontiguous mezczyzna/1.
:- discontiguous kobieta/1.

%rodzice
ojciec(jan, adam).
ojciec(jan, pawel).
ojciec(jan, anna).
matka(maria, adam).
matka(maria, pawel).
matka(maria, anna).

malzenstwo(jan, maria).

%płeć
mezczyzna(jan).
mezczyzna(adam).
mezczyzna(pawel).
kobieta(maria).
kobieta(anna).

% rodzice
ojciec(marek, kacper).
matka(olga, kacper).


malzenstwo(marek, olga).

%płeć
mezczyzna(marek).
mezczyzna(kacper).
kobieta(olga).

%dziadkowie
ojciec(stanislaw, maria).
ojciec(stanislaw, olga).
matka(barbara, maria).
matka(barbara, olga).
mezczyzna(stanislaw).
kobieta(barbara).

%rodzic
rodzic(X, Y) :- ojciec(X, Y).
rodzic(X, Y) :- matka(X, Y).

%przodek
przodek(X, Y) :- rodzic(X, Y).
przodek(X, Y) :-
  rodzic(X, Z),
  przodek(Z, Y).


%malzenstwo
malzenstwo(X, Y) :-
    malzenstwo(Y, X).

% rodzenstwo
brat(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y,
  mezczyzna(X).

siostra(X, Y) :-
  rodzic(Z, X),
  rodzic(Z, Y),
  X \= Y,
  kobieta(X).


%kuzynostwo
rodzenstwo(X, Y) :-
  brat(X, Y).
rodzenstwo(X, Y) :-
  siostra(X, Y).

%kuzynostwo
kuzyn(X, Y) :-
  rodzic(Z, X),
  rodzic(V, Y),
  rodzenstwo(Z, V).

% Przykład: kuzyn(kacper, adam).
% Przykład: siostra(maria, olga).
% Przykład: brat(pawel, adam).
% Przykład: przodek(stanislaw, adam).

### Zadanie 2: System rezerwacji lotów

**Cel zadania:** Stwórz i zaimplementuj bazę faktów dla systemu rezerwacji lotów, który umożliwia zarządzanie lotniskami, połączeniami między nimi oraz cenami biletów. Opracuj reguły, które pozwolą na zapytania o bezpośrednie połączenia, koszt podróży oraz znalezienie najkrótszej trasy między lotniskami.


In [ ]:
% Fakty dotyczące lotnisk
lotnisko(warszawa).
lotnisko(krakow).
lotnisko(gdansk).
lotnisko(wroclaw).
lotnisko(poznan).
lotnisko(lodz).
lotnisko(szczecin).
lotnisko(katowice).

% Fakty dotyczące połączeń i cen biletów
polaczenie(warszawa, krakow, 300).
polaczenie(krakow, gdansk, 200).
polaczenie(warszawa, gdansk, 500).
polaczenie(gdansk, wroclaw, 400).
polaczenie(warszawa, wroclaw, 350).
polaczenie(krakow, wroclaw, 150).
polaczenie(warszawa, poznan, 250).
polaczenie(poznan, wroclaw, 100).
polaczenie(wroclaw, lodz, 200).
polaczenie(lodz, katowice, 150).
polaczenie(katowice, szczecin, 300).
polaczenie(szczecin, gdansk, 450).
polaczenie(warszawa, katowice, 400).
polaczenie(krakow, szczecin, 600).

% Reguła sprawdza, czy istnieje bezpośrednie połączenie
polaczenie(X, Y) :- polaczenie(X, Y, _).

% Reguła określa koszt podróży między lotniskami X i Y
koszt_podrozy(X, Y, C) :- polaczenie(X, Y, C).


% Reguła najkrotsza_droga(X, Y, Trasa, Koszt) - wyznacza najkrótszą trasę
najkrotsza_droga(X, Y, Trasa, Koszt) :-
    polaczenie(X, Y, Koszt),
    Trasa = [X, Y].

najkrotsza_droga(X, Y, Trasa, Koszt) :-   % przykład:najkrotsza_droga(warszawa, gdansk, Trasa, Koszt).
    polaczenie(X, Z, Koszt1),
    najkrotsza_droga(Z, Y, Trasa1, Koszt2),
    \+ member(X, Trasa1), % Unikam cykli, oraz użyłem member - sprawdza czy x znajduje się we wskazanej liscie.
    Koszt is Koszt1 + Koszt2,
    append([X], Trasa1, Trasa).

% Reguła najkrotsza_droga
najkrotsza_droga(X, Y) :-
    najkrotsza_droga(X, Y, Trasa, Koszt),
    write('Najkrotsza trasa: '), write(Trasa), nl,
    write('Koszt: '), write(Koszt).

### Zadanie 3: System zarządzania magazynem

**Cel zadania:** Stwórz i zaimplementuj bazę faktów dla systemu zarządzania magazynem. Baza danych powinna zawierać informacje o produktach, ich ilościach oraz lokalizacji w magazynie. Opracuj reguły, które umożliwią zarządzanie dostępnością produktów, ich przenoszeniem oraz uzupełnianiem stanu.

In [ ]:
% Predykat produkt/3 musi być dynamiczny, ponieważ jego dane mogą być zmieniane w trakcie działania programu.
:- dynamic produkt/3.
% Fakty o produktach
produkt(tv, 10, sektor_a).
produkt(telefon, 30, sektor_b).
produkt(pralka, 20, sektor_c).
produkt(lodowka, 25, sektor_b).
produkt(kamera, 10, sektor_c).
produkt(mikrofon, 19, sektor_a).
%dostępność produktu
dostepny_produkt(X) :-
    produkt(X, N, _),
    N > 0,
	write('Produkt Dostepny').
%przenoszenie produkt do innego sektoru
przenies_produkt(X, Y) :-
    produkt(X, N, OldY),
    retract(produkt(X, N, OldY)), % usuwa
    assertz(produkt(X, N, Y)), % dodaje
	write(produkt(X, N, Y)). % nowe miejsce

%dodawanie ilosci porduktu
uzupelnij_stan(X, N) :-
    produkt(X, N1, Y),
    N2 is N1 + N, % dodaje do siebie stara wartosc z nowa
    retract(produkt(X, N1, Y)),
	assertz(produkt(X, N2, Y)),
	write(produkt(X, N2, Y)). %nowa ilosc


% Przykład: uzupelnij_stan(telefon, 10).
% % Przykład: dostepny_produkt(tv).
% Przykład: przenies_produkt(tv, sektor_c).

### Zadanie 4: System planowania trasy kuriera

**Cel zadania:** Zaprojektuj i wdroż w Prologu bazę faktów dla systemu planowania trasy kuriera. Baza danych powinna zawierać informacje o miastach, połączeniach drogowych między nimi oraz czasie przejazdu. Opracuj reguły umożliwiające zapytania o dostępność dróg, obliczanie czasu przejazdu oraz wyznaczanie najkrótszej trasy.

In [ ]:
% Miasta
miasto(warszawa).
miasto(krakow).
miasto(poznan).
miasto(gdansk).
miasto(wroclaw).
miasto(bialystok).
miasto(rzeszow).

% Drogi i czas przejazdu (w godzinach)
droga(warszawa, krakow, 4).
droga(krakow, poznan, 3).
droga(warszawa, poznan, 7).
droga(poznan, wroclaw, 2).
droga(wroclaw, gdansk, 6).
droga(gdansk, poznan, 4).
droga(bialystok, rzeszow, 3).
droga(warszawa, bialystok, 2).
droga(krakow, rzeszow, 1).

% Drogi działają w obie strony
droga_symmetric(X, Y, T) :- droga(X, Y, T); droga(Y, X, T). % Droga działa w obu kierunkach.

% Sprawdzanie, czy jest droga między miastami
droga(X, Y) :- droga_symmetric(X, Y, _). % Sprawdza, czy jest droga między X a Y.

% Obliczanie czasu przejazdu
czas_przejazdu(X, Y, T) :-
    droga_symmetric(X, Y, T). % Jeśli jest bezpośrednia droga, zwraca jej czas.
czas_przejazdu(X, Y, T) :-
    droga_symmetric(X, Z, T1), % Znajduje drogę pośrednią.
    czas_przejazdu(Z, Y, T2), % Oblicza czas od punktu pośredniego.
    T is T1 + T2. % Sumuje czas.

% Znajdowanie najkrótszej trasy
najkrotsza_trasa(X, Y) :-
    findall((T, Trasa), sciezka(X, Y, [X], Trasa, T), Sciezki), % Szuka wszystkich tras i czasów.
    sort(Sciezki, [(Czas, Trasa)|_]), % Wybiera najkrótszą trasę.
    write('Najkrótsza trasa: '), write(Trasa), nl,
    write('Czas przejazdu: '), write(Czas), nl.

% Reguła pomocnicza do znajdowania trasy
sciezka(X, Y, Odwiedzone, [X, Y], T) :-
    droga_symmetric(X, Y, T),
    \+ member(Y, Odwiedzone). % Sprawdza, czy miasto docelowe nie było odwiedzone.

sciezka(X, Y, Odwiedzone, [X|Reszta], T) :-
    droga_symmetric(X, Z, T1), % Szuka drogi do miasta pośredniego.
    Z \= Y,
    \+ member(Z, Odwiedzone), % Sprawdza, czy miasto pośrednie nie było odwiedzone.
    sciezka(Z, Y, [Z|Odwiedzone], Reszta, T2), % Szuka dalszej trasy.
    T is T1 + T2.


### Zadanie 5: System rekomendacji książek

**Cel zadania:** Stwórz system rekomendacji książek, który uwzględnia preferencje użytkowników, gatunki literackie oraz oceny książek. Twój system powinien być w stanie rekomendować książki danego gatunku użytkownikowi na podstawie jego preferencji oraz wcześniej przyznanych ocen.

In [ ]:
:- dynamic ocena/3.

% Fakty
ksiazka('Zbrodnia i kara', dostojewski, powiesc).
ksiazka('Poezje wybrane', szymborska, poezja).
ksiazka('Quo Vadis', sienkiewicz, powiesc).
ksiazka('Proces', kafka, powiesc).
ksiazka('Ballady i romanse', mickiewicz, poezja).
ksiazka('Wladca Pierscieni', tolkien, powiesc).
ksiazka('Egzorcysta', blatty, horror).
ksiazka('Harry Potter', rowling, fantasy).
ocena(uzytkownik1, 'Zbrodnia i kara', 4).
ocena(uzytkownik1, 'Wladca Pierscieni', 5).
ocena(uzytkownik2, 'Poezje wybrane', 5).
ocena(uzytkownik2, 'Harry Potter', 3).
ocena(uzytkownik3, 'Egzorcysta', 2).
ocena(uzytkownik4, 'Quo Vadis', 1).
preferencje(uzytkownik1, powiesc).
preferencje(uzytkownik2, poezja).
preferencje(uzytkownik3, fantasy).
preferencje(uzytkownik4, horror).

% ocenianie
dodaj_ocene(X, Ksiazka, Ocena) :-
    assertz(ocena(X, Ksiazka, Ocena)),
    write('Pomyslnie dodano: '), write(ocena(X, Ksiazka, Ocena)).


%usuwanie ocen
usun_ocene(X, Ksiazka) :-
    retract(ocena(X, Ksiazka, _)),
    write('Pomyslnie usunieto ocene').


% Polecanie ksiazki
polecane_ksiazki(X, Gatunek) :-
    findall(
        Tytul, (ksiazka(Tytul, _, Gatunek),
            preferencje(X, Gatunek),
            ocena(X, Tytul, Ocena),
            Ocena >= 1),
        PolecaneKsiazki),
    write(PolecaneKsiazki).
% Przykład: polecane_ksiazki(uzytkownik1, powiesc).
% Przykład: dodaj_ocene(uzytkownik2, 'Ballady i romanse', 4).
% Przykład: usun_ocene(uzytkownik2, 'Poezje wybrane').

%findall zawiera tytuły ksiazek
